# Tim Allen: Design and Research Craftsman

### "What’s the least amount of fidelity you can give to an idea that allows people to understand that it’s a great idea?" —via [Finding Our Way](https://findingourway.design/2023/04/12/38-the-craft-led-design-executive-ft-tim-allen/)

### We think across the food, cooking, and groceries space with the Tim Allen

Instructions for getting cozy with this AI recipe are on the [GitHub page's README](https://aka.ms/CAIK-repo). 

# 🧑‍🍳 Recipe for AI-driven 

- ~500 tokens from COMPLETION Pre-trained Foundation Model
- borrow a little multi-modal AI model energy from GPT-4V

> [!IMPORTANT]
> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

# Gather the core tools 🧰

When running the following cell, if asked to "select your kernel" (note this will be referring to the Jupyter notebook's kernel and not Semantic Kernel) then choose `.NET Interactive` from the available menu options.

In [ ]:
// Load some helper functions, e.g. to load values from settings.json
#!import ../config/Settings.cs 

#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta3"
#r "nuget: System.Linq.Async, 6.0.1"

# Fire up a kernel with TWO models 🔥

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

var memoryBuilder = new MemoryBuilder();
var kernelBuilder = new KernelBuilder();

// gpt-3.5-turbo is used by default
model = "gpt-4";

Console.WriteLine($"Using 🧱 Model: {model}");

if (useAzureOpenAI)
{
    memoryBuilder.WithAzureTextEmbeddingGenerationService("text-embedding-ada-002", azureEndpoint, apiKey);
    kernelBuilder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
}
else
{
    memoryBuilder.WithOpenAITextEmbeddingGenerationService("text-embedding-ada-002", apiKey);
    kernelBuilder.WithOpenAIChatCompletionService(model, apiKey, orgId);
}

memoryBuilder.WithMemoryStore(new VolatileMemoryStore());

var kernel = kernelBuilder.Build();
var memory = memoryBuilder.Build();

## A 👁️ multi-modal model might be fun to add to the mix

Let's use a photo by <a href="https://unsplash.com/@nicotitto?utm_content=creditCopyText&utm_medium=referral&utm_source=unsplash">nrd</a> on <a href="https://unsplash.com/photos/bunch-of-vegetables-D6Tu_L3chLE?utm_content=creditCopyText&utm_medium=referral&utm_source=unsplash">Unsplash</a> and ask GPT-4V what's in the image.

![](grocerystore_nicottito.png)

General access to GPT-4V isn't available by API yet, so we're just simulating it here.

In [ ]:
var vizResult = @"
The grocery store has only the following produce available:

* Lettuce
* Green zucchinis
* Yellow squash
* Cucumbers
* A variety of bell peppers 
* Cabbages
* Broccoli
* Brussels sprouts
* Parsley
";

# Generate embeddings for later 🧲 gathering

In [ ]:
const string MemoryCollectionName = "preferences";

string[] notes = { 
    "for breakfast, meals with eggs are preferred and low salt",
    "for lunch, a meal with less than 300 calories is best",
    "for dinner, definitely offer dessert"
};

for (int i = 0; i < notes.Length; i++)
{
    await memory.SaveInformationAsync(MemoryCollectionName, id: $"note{i}", text: notes[i]);
}

# You can always 🤔 look up what your visitor prefers

In [ ]:
var q = "breakfast";
var response = await memory.SearchAsync(MemoryCollectionName, q).FirstOrDefaultAsync();
var mealPreference = response?.Metadata.Text;
Console.WriteLine($"{q}\n> {mealPreference}");

## Now we have enough information to get 🧑‍🍳 cooking

Let's add the inventory of what's in my fridge (that we could get from a 🔌 Plugin) to what we know about our world. And let's use the image snapshot of what we can get at our local grocer.

In [ ]:
var inMyFridgeAndPantry = @"
milk, eggs, butter, chicken, cheese, yogurt, salt, pepper, sugar, olive oil, flour
";

Console.WriteLine($"Meal preference: {mealPreference}");
Console.WriteLine($"In my fridge and pantry:\n{inMyFridgeAndPantry}");
Console.WriteLine($"Available at my grocer: {vizResult}");

## Let's 🤔 determine what to make ...

In [ ]:
using Microsoft.SemanticKernel.Orchestration;

var pluginsSKDirectory = "plugins-sk";

var myFunctions = kernel.ImportSemanticFunctionsFromDirectory(pluginsSKDirectory, "CookingForReal");

var myContext = new ContextVariables(); 

myContext.Set("grocer", vizResult); 
myContext.Set("available", inMyFridgeAndPantry);
myContext.Set("mealtype", mealPreference); 

var myResult = await kernel.RunAsync(myContext,myFunctions["MealSuggester"]);
var mealString = myResult.GetValue<string>();

Console.WriteLine(mealString);

## Let's go 🛒 shopping 

In [ ]:
using Microsoft.SemanticKernel.Orchestration;

var pluginsSKDirectory = "plugins-sk";

var myFunctions = kernel.ImportSemanticFunctionsFromDirectory(pluginsSKDirectory, "CookingForReal");

var myContext = new ContextVariables(); 

myContext.Set("grocer", vizResult); 
myContext.Set("available", inMyFridgeAndPantry);
myContext.Set("themeal", mealString); 

var myResult = await kernel.RunAsync(myContext,myFunctions["ShoppingSuggester"]);
var resultString = myResult.GetValue<string>();

Console.WriteLine(resultString);

# Plate the partially finished meal 🍽️

Let's ready an image generation model ...

In [ ]:
#r "nuget: SkiaSharp, 2.88.3"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI.ImageGeneration; 
using Microsoft.SemanticKernel.AI.Embeddings;
using Microsoft.SemanticKernel.AI.Embeddings.VectorOperations;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

// Load OpenAI credentials from config/settings.json
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

// Configure the three AI features: text embedding (using Ada), text completion (using DaVinci 3), image generation (DALL-E 2)
var builder = new KernelBuilder();

if(useAzureOpenAI)
{
    builder.WithAzureOpenAIImageGenerationService(azureEndpoint, apiKey);
}
else
{
    builder.WithOpenAIImageGenerationService(apiKey, orgId);
}
   
var kernel = builder.Build();

// Get AI service instance used to generate images
var dallE = kernel.GetService<IImageGeneration>();

And let's render that meal we were talking about ...

In [ ]:
#!import ../config/SkiaUtils.cs

using System;
using System.Threading.Tasks;
using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
using System.IO;
using System.Net.Http;

HttpClient httpClient = new HttpClient();

async Task DownloadImageAsync(string uri, string localPath)
{
    using (HttpResponseMessage response = await httpClient.GetAsync(uri))
    {
        response.EnsureSuccessStatusCode();

        using (Stream contentStream = await response.Content.ReadAsStreamAsync(), 
                fileStream = new FileStream(localPath, FileMode.Create, FileAccess.Write, FileShare.None))
        {
            await contentStream.CopyToAsync(fileStream);
        }
    }
}

Console.WriteLine(mealString);

var imageDescription = mealString;
var image = await dallE.GenerateImageAsync(imageDescription, 256, 256);

Console.WriteLine(imageDescription);
Console.WriteLine("Image URL: " + image);
string uniqueFileName = "abc-" + Guid.NewGuid().ToString() + ".png";
Console.WriteLine(uniqueFileName);
await DownloadImageAsync(image,uniqueFileName);

You can look at it inline too:

In [ ]:
await SkiaUtils.ShowImage(image, 256, 256);